In [1]:
from util import *
from search import *
from prototype import *
from random import shuffle

In [2]:
seqs = load_fasta_records_as_str("uniprot_sprot.fasta")
shuffle(seqs)
seqs = seqs[:1000]
peps = collapse_second_order_list(map(digest_trypsin,seqs))
peps = list(filter(lambda pep: 'X' not in pep, peps))
specs = list(map(generate_spectrum_and_list_mz,peps))

In [3]:
tolerance = 2 * AVERAGE_MASS_DIFFERENCE
alphabet = AMINO_MASS_MONO
for spec in specs:
    assert search(specs[0],tolerance,alphabet)[0] == search_overlap(specs[0],tolerance,alphabet) == search_overlap2(specs[0],tolerance,alphabet)

KeyboardInterrupt: 

In [ ]:
from time import time
def duration(fn, specs, tolerance, alphabet):
    init_t = time()
    for spec in specs:
        fn(spec,tolerance,alphabet)
    return time() - init_t

In [ ]:
duration(search,specs,tolerance,alphabet)

In [ ]:
duration(search_overlap,specs,tolerance,alphabet)

In [ ]:
duration(search_overlap2,specs,tolerance,alphabet)